In [1]:
%load_ext autoreload
%autoreload 2

from model import *



from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

In [2]:
x = torch.randn((1, 3, 4, 4))

# x = torch.tensor(
    # [[0, 1., 0],     
    # [2, 0, 0],
    # [0, 0, 1]])

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

x, y

(tensor([[[[-2.5897, -0.6237,  0.3294,  0.1079],
           [ 1.8352,  1.5638,  1.3405,  0.4122],
           [-0.2594, -0.4289, -1.2603,  0.6184],
           [ 1.2756,  0.6002,  0.4382, -0.1728]],
 
          [[-1.1410,  1.3268, -0.8286, -0.2027],
           [ 1.3757, -0.3756,  0.3548,  1.8617],
           [ 1.1493, -1.7133, -0.0535,  0.9079],
           [ 0.7819,  0.4096, -0.4855, -0.6761]],
 
          [[-0.7933, -1.2995,  2.7724,  0.8352],
           [-1.4125,  0.0078,  0.7490, -0.0453],
           [ 0.6314,  0.0235,  1.1272,  0.1205],
           [-0.3147,  0.0366, -2.9666,  0.5121]]]]),
 tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]]]))

In [3]:
conv = nn.Conv2d(3, 4, 2)
expected = conv(y)
expected

tensor([[[[ 0.0140,  0.0140,  0.0140],
          [ 0.0140,  0.0140,  0.0140],
          [ 0.0140,  0.0140,  0.0140]],

         [[-0.1729, -0.1729, -0.1729],
          [-0.1729, -0.1729, -0.1729],
          [-0.1729, -0.1729, -0.1729]],

         [[ 0.9094,  0.9094,  0.9094],
          [ 0.9094,  0.9094,  0.9094],
          [ 0.9094,  0.9094,  0.9094]],

         [[ 0.6292,  0.6292,  0.6292],
          [ 0.6292,  0.6292,  0.6292],
          [ 0.6292,  0.6292,  0.6292]]]])

In [5]:
# torch.nn.Unfold(kernel_size=2)(x)

kernel_size = (2, 2)
out_channels = 4

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(y, kernel_size=kernel_size)
# unfolded

wxb = conv.weight.view(out_channels, -1) @ unfolded + conv.bias.view(1, -1, 1)
actual = wxb.view(1, out_channels, y.shape[2] - kernel_size[0] + 1, y.shape[3] - kernel_size[1]+ 1)
torch.testing.assert_allclose(actual, expected)

### Testing our Conv2d Layer 

Forward pass

In [14]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
x2 = my_conv.forward(x)
x2.shape

torch.Size([1, 4, 3, 3])

In [15]:
my_conv.input_x.shape

torch.Size([1, 12, 9])

In [13]:
my_conv.backward(x2)

AttributeError: 'Conv2d' object has no attribute 'prev_x'

In [7]:
torch.testing.assert_allclose(my_conv.forward(x), torch.nn.functional.conv2d(x, my_conv.weight, my_conv.bias))

Backward pass

In [8]:
x2.shape

torch.Size([1, 4, 3, 3])

In [10]:
w = my_conv.weight
w.shape

torch.Size([4, 3, 2, 2])

In [9]:
(x2.view(3, -1) @ w.view(4, -1).t()).shape

NameError: name 'w' is not defined

In [ ]:
in_unfolded = unfold(x, kernel_size=2)
out_unfolded = my_conv.weight.view(4, -1) @ in_unfolded + my_conv.bias.view(1, -1, 1)
ouput = out_unfolded.view(x.shape[0], self.output_channels, tensor.shape[2]-self.kernel_size[0]+1, tensor.shape[3]-self.kernel_size[1]+1)

In [164]:
my_conv.backward(x2)

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1

### Testing our TransposeConv2d layer

Forward pass

In [11]:
my_t_conv = TransposeConv2d(3, 5, 2)
y_expected = my_t_conv.forward(x)
y_expected.shape

torch.Size([1, 5, 5, 5])

In [12]:
torch.testing.assert_allclose(my_t_conv.forward(x), torch.nn.functional.conv_transpose2d(x, my_t_conv.weight, my_t_conv.bias))

Backward pass

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])